# Processo de Decisao de Markov

<br><br>

Nesse Notebook, vamos introduzir o processo de decisao de Markov, vamos utilizar a biblioteca gym do OpenAI para isso
<br><br><br>


In [ ]:
import gym
import numpy as np
from IPython import display
from matplotlib import pyplot as plt

from envs import Maze
%matplotlib inline

##### Construindo o Ambiente: Maze()

Esse metodo retorna uma instance da classe Maze. Nao precisaremos mexer por enquanto

In [ ]:
env = Maze()

##### env.reset()

Reiniciar o ambiente de labirinto para que possamos observar o agente


In [ ]:
initial_state = env.reset()
print(f"O novo episodio vai iniciar no estado: {initial_state}")

##### env.render()

Gera a imagem do atual estado que se encontra o labirinto em formato np.ndarray. 

In [ ]:
frame = env.render(mode='rgb_array')
plt.axis('off')
plt.title(f"State: {initial_state}")
plt.imshow(frame)

##### env.step()

Esse metodo aplica a acao selecionada pelo agente no ambiente a fim de modifica-lo. Em resposta, o ambiente retorna uma tupla de quatro objetos

- O proximo estado
- O pagamento obtido
- Se a tarefa foi completa (T/F)
- Se ha qualquer outra informacao relevante em forma de dicionario 


In [ ]:
action = 2
next_state, reward, done, info = env.step(action)
print(f"Após mover 1 linha para baixo, o agente esta no estado: {next_state}")
print(f"Após mover 1 linha para baixo, temos um pagamento de: {reward}")
print("Após mover 1 linha para baixo, a tarefa ", "" if done else "não ", "acabou")

##### Renderizar o novo estado

In [ ]:
frame = env.render(mode='rgb_array')
plt.axis('off')
plt.title(f"State: {next_state}")
plt.imshow(frame)

##### env.close()

Finalizar a tarefa e fechar o ambiente, liberando os recursos computacionais.

In [ ]:
env.close()

## Labirinto: encontre a saída.

Vamos utilizar esse ambiente para os proximos algoritmos: PD, Monte-Carlo e TD. Esse ambiente é dito apropriado para aprender o basico de RL pois:

- Possui poucos estados (25)
- Transicoes entre estados sao deterministicos ($p(s', r| s, a) = 1$)
- Todos os pagamentos sao os mesmos (-1) ate que o episodio conclua. Isso facilita o estudo de funcoes de $V(S)$ e $Q(S,A)$

Atraves desse ambiente, vamos revisar os conceitos vistos anteriormente:

- Estados
- Acoes
- Trajetorias
- Retornos e pagamentos
- Politica

O ambiente é um labirinto 5x5 no qual a meta do agente é encontrar a saída do labirinto, localizada no canto direito de baixo, na celula (4,4), colorida de verde claro.

Para chegar a saida, o agente pode tomar quatro diferente acoes: mover para cima, baixo, direita e esquerda.

###### Criar o ambiente

In [ ]:
env = Maze()

##### Espaço dos Estados

Os estados consistem em uma tuple com dois numeros inteiros no intervalo $[0,4]$ representando a linha e a coluna em que o agente está atualmente localizado:
<br>
<br>

\begin{equation}
    s = (row, column) \;\\
    row, column \in \{0,1,2,3, 4\}
\end{equation}
<br>
<br>
O espaço de busca possui 25 elementos (todas a possiveis combinacoes de linhas e colunas)

\begin{equation}
    Rows \times Columns \;\\
    S = \{(0, 0), (0, 1), (1, 0), ...\}
\end{equation}

Informacao sobre o espaço de busca está armazenado na variavel env.observation_space.

In [ ]:
print(f"O estado inicial é: {env.reset()}")
print(f"O espaço de busca é do tipo: {env.observation_space}")

##### Ações e espaço de ações

Neste ambiente, existem quatro diferentes ações que sāo reprensentados por numeros inteiros:

\begin{equation}
a \in \{0, 1, 2, 3\}
\end{equation}

- 0 -> mover para cima
- 1 -> mover para direita
- 2 -> mover para baixo
- 3 -> mover para esquerda

Para executar uma acao, passe como um argumento para o metodo env.step


In [ ]:
print(f"Um exemplo de ação valida é: {env.action_space.sample()}")
print(f"O espaço de ação é do tipo: {env.action_space}")

##### Trajetorias e episodios

Uma Trajetoria é a sequencia gerada quando se move de um estado arbitrario para o outro

\begin{equation}
  \tau = S_0, A_0, R_1, S_1, A_1, ... R_N, S_N,
\end{equation}

Vamos gerar uma trajetoria de tres estados abaixo

In [ ]:
env = Maze()
state = env.reset()
trajectory = []
for _ in range(3):
    action = env.action_space.sample()
    next_state, reward, done, extra_info = env.step(action)
    trajectory.append([state, action, reward, done, next_state])
    state = next_state
env.close()

print(f"Você gerou a sua primeira trajetoria:\n{trajectory}")

Um episodio é a trajetoria que vai do estado inicial do processo até o final 

\begin{equation}
  \tau = S_0, A_0, R_1, S_1, A_1, ... R_T, S_T,
\end{equation}
Aonde T é o estado terminal

Vamos gerar um episodio completo abaixo:

In [ ]:
env = Maze()
state = env.reset()
episode = []
done = False
while not done:
    action = env.action_space.sample()
    next_state, reward, done, extra_info = env.step(action)
    episode.append([state, action, reward, done, next_state])
    state = next_state
env.close()

print(f"Você gerou o seu primeiro episodio:\n{episode}")

##### Pagamentos e retornos

Um pagamento é o feedback numero que o ambiente gera quando o agente toma uma ação *a* num estado *s*:

\begin{equation}
    r = r(s, a)
\end{equation}

Vamos gerar um pagamento atraves do ambiente:

In [ ]:
env = Maze()
state = env.reset()
action = env.action_space.sample()
_, reward, _, _ = env.step(action)
print(f"Obteve-se um pagamento de {reward} tomando a ação {action} no estado {state}")

O retorno associado com o momento no tempo *t* é a soma (descontada) dos pagamentos que o agente obtém daquele momento.
Vamos calcular $G_0$, o retorno no inicio do episodio:

\begin{equation}
    G_0 = R_1 + \gamma R_2 + \gamma^2 R_3 + ... + \gamma^{T-1} R_T
\end{equation}

Assumindo que o fator de desconto $\gamma = 0.99$:


In [ ]:
env = Maze()
state = env.reset()
done = False
gamma = 0.99
G_0 = 0
t = 0
while not done:
    action = env.action_space.sample()
    _, reward, done, _ = env.step(action)
    G_0 += gamma ** t * reward
    t += 1
env.close()

print(
    f"""Tomou-se {t} movimentos para encontrar a saída, 
    e cada pagamento r(s,a)=-1, entao o retorno se equivale a {G_0}""")

##### Politica

Uma politica é uma função $\pi(a|s) \in [0, 1]$ que dá a probabilidade de uma ação dado um estado. A função possui como entrada um estado e uma ação e retorna um numero de ponto fluante entre $[0,1]$

já que na pratica vamos precisar computar as probabilidades de todas as acoes, vamos representar a politica como uma funcao que toma um estado como argumento e retorna as probabilidades associadas com cada acao, um exemplo seria:

[0.5, 0.3, 0.1]

In [ ]:
def random_policy(state):
    return np.array([0.25] * 4)

## Simular um episodio utilizando a politica aleatoria

###### Criar e reiniciar o ambiente

In [ ]:
env = Maze()
state = env.reset()

#### Computar $p(a|s) \; \forall a \in \{0, 1, 2, 3\}$

In [ ]:
action_probabilities = random_policy(state)

In [ ]:
objects = ('Up', 'Right', 'Down', 'Left')
y_pos = np.arange(len(objects))

plt.bar(y_pos, action_probabilities, alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('P(a|s)')
plt.title('Politica Estocastica')
plt.tight_layout()

plt.show()

#### Usar a politica para simular um episodio

In [ ]:
env.reset()
done = False
img = plt.imshow(env.render(mode='rgb_array')) 
while not done:
    action = np.random.choice(range(4), 1, p=action_probabilities)
    _, _, done, _ = env.step(action)
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
env.close()